In [1]:
#Import modules
import psycopg2
import pandas as pd
import numpy as np

#Import password for postgres
from passwords import password 

In [2]:
#Open a connection to postgres and create a connection object
conn = psycopg2.connect(host='localhost', dbname='glass', user='postgres', password=password)
#Create a cursor object to execute commands
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS observation_data')
cur.execute('''
    CREATE TABLE observation_data(
        RI float,
        NA float,
        MG float,
        AL float,
        SI float,
        K float,
        CA float,
        BA float,
        FE float,
        TYPE integer)
''')

#Commit changes 
conn.commit()

In [3]:
#Create a cursor object to execute commands
cur = conn.cursor()

#Open the created csv file
with open('glass (1).csv', 'r') as i:
    #Skip header row
    next(i) 
    
    #Call the created unemployment table and copy data from csv file into table
    cur.copy_from(i, 'observation_data', sep=',', null='')

#Commit changes
#cur.execute('ALTER TABLE observation_data DROP COLUMN TYPE')
conn.commit()

In [4]:
cur = conn.cursor()

cur.execute('''
    ALTER TABLE observation_data RENAME TO old_glass;
    CREATE TABLE observation_data (
        OBSERVATION_ID SERIAL PRIMARY KEY,
        RI float,
        NA float,
        MG float,
        AL float,
        SI float,
        K float,
        CA float,
        BA float,
        FE float,
        TYPE integer);
    INSERT INTO observation_data (RI, NA, MG, AL, SI, K, CA, BA, FE, TYPE) SELECT * from old_glass;
    DROP TABLE old_glass;''')

conn.commit()